[Optimised Modes](https://openpyxl.readthedocs.io/en/latest/optimized.html)


## Read-only mode

Sometimes, you will need to open or write extremely large XLSX files,
and the common routines in openpyxl won't be able to handle that load.
Fortunately, there are two modes that enable you to read and write unlimited
amounts of data with (near) constant memory consumption.

Introducing `openpyxl.worksheet._read_only.ReadOnlyWorksheet`::

In [1]:
from openpyxl import load_workbook

wb = load_workbook(filename='large_file.xlsx', read_only=True)
ws = wb['big_data']

for row in ws.rows:
    for cell in row:
        print(cell.value)

# Close the workbook after reading
wb.close()

None
B
C
D
E
F
G
2
=B$1&$A2
=C$1&$A2
=D$1&$A2
=E$1&$A2
=F$1&$A2
=G$1&$A2
3
=B$1&$A3
=C$1&$A3
=D$1&$A3
=E$1&$A3
=F$1&$A3
=G$1&$A3
4
=B$1&$A4
=C$1&$A4
=D$1&$A4
=E$1&$A4
=F$1&$A4
=G$1&$A4
5
=B$1&$A5
=C$1&$A5
=D$1&$A5
=E$1&$A5
=F$1&$A5
=G$1&$A5
6
=B$1&$A6
=C$1&$A6
=D$1&$A6
=E$1&$A6
=F$1&$A6
=G$1&$A6
7
=B$1&$A7
=C$1&$A7
=D$1&$A7
=E$1&$A7
=F$1&$A7
=G$1&$A7
8
=B$1&$A8
=C$1&$A8
=D$1&$A8
=E$1&$A8
=F$1&$A8
=G$1&$A8
9
=B$1&$A9
=C$1&$A9
=D$1&$A9
=E$1&$A9
=F$1&$A9
=G$1&$A9
10
=B$1&$A10
=C$1&$A10
=D$1&$A10
=E$1&$A10
=F$1&$A10
=G$1&$A10
11
=B$1&$A11
=C$1&$A11
=D$1&$A11
=E$1&$A11
=F$1&$A11
=G$1&$A11
12
=B$1&$A12
=C$1&$A12
=D$1&$A12
=E$1&$A12
=F$1&$A12
=G$1&$A12
13
=B$1&$A13
=C$1&$A13
=D$1&$A13
=E$1&$A13
=F$1&$A13
=G$1&$A13
14
=B$1&$A14
=C$1&$A14
=D$1&$A14
=E$1&$A14
=F$1&$A14
=G$1&$A14
15
=B$1&$A15
=C$1&$A15
=D$1&$A15
=E$1&$A15
=F$1&$A15
=G$1&$A15
16
=B$1&$A16
=C$1&$A16
=D$1&$A16
=E$1&$A16
=F$1&$A16
=G$1&$A16
17
=B$1&$A17
=C$1&$A17
=D$1&$A17
=E$1&$A17
=F$1&$A17
=G$1&$A17
18
=B$1&$A18
=C$1&$A18
=D$1&$A1

> warning:
>
>   * `openpyxl.worksheet._read_only.ReadOnlyWorksheet` is read-only
>
>    * Unlike a normal workbook, a read-only workbook will use lazy loading.
>     The workbook must be explicitly closed with the :func:`close()` method.

Cells returned are not regular `openpyxl.cell.cell.Cell` but
`openpyxl.cell._read_only.ReadOnlyCell`.


### Worksheet dimensions


Read-only mode relies on applications and libraries that created the file
providing correct information about the worksheets, specifically the used
part of it, known as the dimensions. Some applications set this incorrectly.
You can check the apparent dimensions of a worksheet using
`ws.calculate_dimension()`. If this returns a range that you know is
incorrect, say `A1:A1` then simply resetting the max_row and max_column
attributes should allow you to work with the file:

In [2]:
ws.reset_dimensions()

## Write-only mode


Here again, the regular `openpyxl.worksheet.worksheet.Worksheet` has been replaced
by a faster alternative, the `openpyxl.worksheet._write_only.WriteOnlyWorksheet`.
When you want to dump large amounts of data make sure you have `lxml` installed.

In [3]:
from openpyxl import Workbook

wb = Workbook(write_only=True)
ws = wb.create_sheet()

# now we'll fill it with 100 rows x 200 columns

for irow in range(100):
    ws.append(['%d' % i for i in range(200)])

# save the file
wb.save('new_big_file.xlsx') # doctest: +SKIP

If you want to have cells with styles or comments then use a `openpyxl.cell.WriteOnlyCell`

In [4]:
from openpyxl import Workbook

wb = Workbook(write_only = True)
ws = wb.create_sheet()

In [5]:
from openpyxl.cell import WriteOnlyCell
from openpyxl.comments import Comment
from openpyxl.styles import Font

cell = WriteOnlyCell(ws, value="hello world")
cell.font = Font(name='Courier', size=36)
cell.comment = Comment(text="A comment", author="Author's Name")
ws.append([cell, 3.14, None])
wb.save('write_only_file.xlsx')

This will create a write-only workbook with a single sheet, and append
a row of 3 cells: one text cell with a custom font and a comment, a
floating-point number, and an empty cell (which will be discarded
anyway).

>  Warning:
>
>   * Unlike a normal workbook, a newly-created write-only workbook
>     does not contain any worksheets; a worksheet must be specifically
>     created with the :func:`create_sheet()` method.
>
>   * In a write-only workbook, rows can only be added with
>     :func:`append()`. It is not possible to write (or read) cells at
>     arbitrary locations with :func:`cell()` or :func:`iter_rows()`.
>
>   * It is able to export unlimited amount of data (even more than Excel can
>     handle actually), while keeping memory usage under 10Mb.
>
>   * A write-only workbook can only be saved once. After
>     that, every attempt to save the workbook or append() to an existing
>     worksheet will raise an :class:`openpyxl.utils.exceptions.WorkbookAlreadySaved`
>     exception.
>
>   * Everything that appears in the file before the actual cell data must be created
>     before cells are added because it must written to the file before then.
>     For example, `freeze_panes` should be set before cells are added.

---
